In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\Medical Chatbot\\Medical-Chatbot-Hugginface\\research'

In [3]:
#Change the current working directory to the parent directory
import os
os.chdir("../")

In [4]:
pip install langchain_community


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [8]:
DATA_PATH="data/"
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [9]:

documents=load_pdf_files(data=DATA_PATH)
print("Length of PDF pages: ", len(documents))

Length of PDF pages:  206


In [10]:
def create_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=create_chunks(extracted_data=documents)
print("Length of Text Chunks: ", len(text_chunks))

Length of Text Chunks:  939


In [12]:
def get_embedding_model():
    embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

embedding_model=get_embedding_model()

c:\Users\nayak\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [14]:
DB_FAISS_PATH="vectorstore/db_faiss"
db=FAISS.from_documents(text_chunks, embedding_model)
db.save_local(DB_FAISS_PATH)

In [15]:
import os

from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
HF_TOKEN=os.environ.get("HF_TOKEN")
HUGGINGFACE_REPO_ID="HuggingFaceH4/zephyr-7b-beta"

In [17]:
CUSTOM_PROMPT_TEMPLATE = """
You are a clinical assistant AI. Use only the information in the context below to answer.

Rules:
- Do not hallucinate treatments or recommendations.
- Do not make assumptions beyond the given content.
- If the answer isn't present, reply: "This information is not available in the current guidelines."

Context:
{context}

Question:
{question}

Answer:
"""

In [18]:
def set_custom_prompt(custom_prompt_template):
    prompt=PromptTemplate(template=custom_prompt_template, input_variables=["context", "question"])
    return prompt

In [19]:
DB_FAISS_PATH="vectorstore/db_faiss"
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db=FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

In [20]:
def load_llm(huggingface_repo_id):
    llm = HuggingFaceEndpoint(
        repo_id=HUGGINGFACE_REPO_ID,
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
        max_new_tokens=512   # ✅ Passed directly, NOT inside model_kwargs
    )
    return llm


In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm=load_llm(HUGGINGFACE_REPO_ID),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt': set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)},
    input_key="question"  # if you're using 'question' instead of 'query'
)

In [24]:
pip install flask

  Using cached flask-3.1.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached flask-3.1.1-py3-none-any.whl (103 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)

   ---------------- ----------------------- 2/5 [click]
   ---------------------------------------- 5/5 [flask]

Note: you may need to restart the kernel to use updated packages.


In [22]:
user_query = input("Write Query Here: ")
response = qa_chain.invoke({'question': user_query})
print("RESULT:", response["result"])
print("SOURCE DOCUMENTS:", response["source_documents"])

KeyboardInterrupt: Interrupted by user

In [ ]:

response = qa_chain.invoke({'question': "what is acne?"})

if not response.get("source_documents"):
        print("This information is not available in the current guidelines.") 

    # Step 2: Check if model response is too generic or admits lack of info
model_answer = response["result"].strip().lower()
if (
        "not mentioned" in model_answer
        or "not available" in model_answer
        or "i don't know" in model_answer
        or "cannot be found" in model_answer
        or "provided context" in model_answer
    ):
    print("This information is not available in the current guidelines.")
else:
      print("RESULT:", response["result"])
print("SOURCE DOCUMENTS:", response["source_documents"])

The model's response is too generic or admits lack of information.
SOURCE DOCUMENTS: [Document(id='605c4618-1d89-4fbd-b21c-cb500e0eab47', metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2019-09-23T11:47:58+05:30', 'author': 'Monica Sharma', 'moddate': '2019-09-23T11:47:58+05:30', 'title': 'Microsoft Word - Final Treatment Guidelines 12.9.19.doc', 'source': 'data\\ICMR_Treatment_Guidelines_2019_Final.pdf', 'total_pages': 206, 'page': 59, 'page_label': '60'}, page_content='The borders in cellulitis are not well demarcated. Rarely organisms like H. influenza and \nPneumococcus may also cause cellulitis.  \n6.2.2 Furunculosis: Furunculosis is a deep infection of the hair follicle leading to \nabscess formation with an accumulation of pus and necrotic tissue. Furuncles appear as \nred, swollen, and tender nodules on hair-bearing parts of the body. The most common \ninfectious agent is S. aureus, but other bacteria may also be causative. Furunculosis oft

In [30]:
pip install gradio

  Using cached gradio-5.31.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.11.11-py3-none-win_amd64.whl.metadata (26 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadat